In [1]:
import joblib
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm
import torch
import torch.nn as nn
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, Subset
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import KFold
import random

In [2]:
loaded_data = joblib.load("train_data.pkl")
targets, data, ids = loaded_data

# 打印某个人的具体信息
for i in tqdm(range(277)):
    if ids[i] == "03d92c9f6f8a":
        print(f"Target: {targets[i]}")
        print(f"{data[i]}")
        print(type(targets[i]))
        print(type(data[i]))

for i in tqdm(range(277)):
    print(ids[i], end=" ")

100%|██████████| 277/277 [00:00<00:00, 69428.84it/s]


Target: [(5928, 13524), (23220, 30276), (40668, 47952), (75756, 82800), (178464, 186564), (196260, 203844), (230820, 241872), (248124, 255060)]
           anglez      enmo      step
0       38.906250  0.080322       0.0
1       29.375000  0.075195       1.0
2       37.218750  0.179077       2.0
3       46.937500  0.092224       3.0
4       60.500000  0.034210       4.0
...           ...       ...       ...
724135 -12.062500  0.038208  724135.0
724136 -15.914062  0.025406  724136.0
724137 -10.859375  0.028107  724137.0
724138  -8.531250  0.027405  724138.0
724139  -9.273438  0.032013  724139.0

[724140 rows x 3 columns]
<class 'list'>
<class 'pandas.core.frame.DataFrame'>


100%|██████████| 277/277 [00:00<00:00, 69399.81it/s]

038441c925bb 03d92c9f6f8a 0402a003dae9 04f547b8017d 05e1944c3818 062cae666e2a 062dbd4c95e6 08db4255286f 0a96f4993bd7 0cd1e3d0ed95 0ce74d6d2106 0cfc06c129cc 0d0ad1e77851 0dee4fda51c3 0ec9fc461819 0ef7d94fde99 0f572d690310 0f9e60a8e56d 10469f6765bf 1087d7b0ff2e 10f8bc1f7b07 12d01911d509 1319a1935f48 137771d19ca2 137b99e936ab 13b4d6a01d27 148471991ffb 154fe824ed87 16fe2798ed0f 1716cd4163b2 1762ab70ec76 188d4b7cd28b 18a0ca03431d 18b61dd5aae8 1955d568d987 1b92be89db4c 1c7c0bad1263 1d4569cbac0f 1e6717d93c1d 1f96b9668bdf 207eded97727 25e2b3dd9c3b 2654a87be968 27f09a6a858f 280e08693c6d 292a75c0b94e 29c75c018220 29d3469bd15d 2b0a1fa8eba8 2b8d87addea9 2cd2340ca14d 2e9ced2c7976 2f7504d0f426 2fbbee1a38e3 2fc653ca75c7 31011ade7c0a 3318a0e3ed6f 33ceeba8918a 3452b878e596 349c5562ee2c 35826366dfc7 361366da569e 3664fe9233f9 3665c86afaf5 390b487231ce 3a9a9dc2cbd9 3aceb17ef7bd 3be1545083b7 3be2f86c3e45 3c336d6ba566 3d53bfea61d6 3df0da2e5966 405df1b41f9f 40dce6018935 416354edd92a 449766346eb1 44a41bba1ee7

In [3]:
train_events = pd.read_csv("./train_events.csv")
num_person = len(train_events["series_id"].unique())
print("The number of total persons is:", num_person)

person_ids = train_events["series_id"].unique()
ids_to_index = {person_id: index for index, person_id in enumerate(person_ids)}
index_to_ids = {index: person_id for index, person_id in enumerate(person_ids)}
print(ids_to_index)
print(index_to_ids)

The number of total persons is: 277
{'038441c925bb': 0, '03d92c9f6f8a': 1, '0402a003dae9': 2, '04f547b8017d': 3, '05e1944c3818': 4, '062cae666e2a': 5, '062dbd4c95e6': 6, '08db4255286f': 7, '0a96f4993bd7': 8, '0cd1e3d0ed95': 9, '0ce74d6d2106': 10, '0cfc06c129cc': 11, '0d0ad1e77851': 12, '0dee4fda51c3': 13, '0ec9fc461819': 14, '0ef7d94fde99': 15, '0f572d690310': 16, '0f9e60a8e56d': 17, '10469f6765bf': 18, '1087d7b0ff2e': 19, '10f8bc1f7b07': 20, '12d01911d509': 21, '1319a1935f48': 22, '137771d19ca2': 23, '137b99e936ab': 24, '13b4d6a01d27': 25, '148471991ffb': 26, '154fe824ed87': 27, '16fe2798ed0f': 28, '1716cd4163b2': 29, '1762ab70ec76': 30, '188d4b7cd28b': 31, '18a0ca03431d': 32, '18b61dd5aae8': 33, '1955d568d987': 34, '1b92be89db4c': 35, '1c7c0bad1263': 36, '1d4569cbac0f': 37, '1e6717d93c1d': 38, '1f96b9668bdf': 39, '207eded97727': 40, '25e2b3dd9c3b': 41, '2654a87be968': 42, '27f09a6a858f': 43, '280e08693c6d': 44, '292a75c0b94e': 45, '29c75c018220': 46, '29d3469bd15d': 47, '2b0a1fa8eba8

In [4]:
class CFG:
    window_overlap = 60
    window_size = 30
    batch_size = 32
    input_dim = 2
    embed_dim = 32
    num_classes = 4 
    num_layers = 4
    nhead = 4
    dim_feedforward = 64
    learning_rate = 1e-3
    epochs = 300
    train_record_steps = 1
    dropout = 0.2
    num_person = 10
    scheduler_step_size = 20
    scheduler_gamma = 0.8
    fold = 8
    weight_decay = 0.001
    model_dir = "./models"

In [5]:
#将每个人的数据分别提取出来

dic_X = {}
dic_y = {}

for i in tqdm(range(min(num_person, CFG.num_person))):

    print("Id:", ids[i])
    person_id = ids[i]

    dic_X[person_id] = []
    dic_y[person_id] = []

    pair_list = targets[i]
    onset_steps = []
    wakeup_steps = []
    for idx in range(len(pair_list)):
        onset_steps.append(pair_list[idx][0])
        wakeup_steps.append(pair_list[idx][1])

    empty_spaces = []
    events = train_events[train_events["series_id"] == person_id]
    last_events = 0
    empty_mark = 0 
    for (idx, row) in events.iterrows():
        if np.isnan(row["step"]):
            empty_mark = 1
        else:
            if row["step"] > last_events and empty_mark == 0:
                last_events = row["step"]
            elif row["step"] > last_events and empty_mark == 1:
                empty_spaces.append((int(last_events) - 360, int(row["step"]) - 360))
                last_events = row["step"]
                empty_mark = 0

    df = data[i]
    labels = []
    for j in range(0, len(df), CFG.window_overlap):
        start = j
        if j + CFG.window_size >= len(df):
            start = len(df) - CFG.window_size
            end = len(df)
            break
        else:
            end = j + CFG.window_size

        flag_empty = 0
        for k in empty_spaces:
            if (k[0] < start < k[1]) or (k[0] < end < k[1]):
                flag_empty = 1
                break
            else:
                continue

        flag_sleep = 0
        for k in range(len(onset_steps)):
            if (onset_steps[k] < start < end < wakeup_steps[k]):
                flag_sleep = 1
                break
            else:
                continue


        chunk = df.iloc[start:end][["anglez", "enmo"]]
        chunk = chunk.to_numpy()

        if flag_empty == 1:
            label = 4
        elif any(start <= num <= end for num in wakeup_steps):
            label = 3
        elif any(start <= num <= end for num in onset_steps):
            label = 2
        elif flag_sleep == 1:
            label = 1
        else:
            label = 0
        labels.append(label)

        if label != 4:
            dic_X[person_id].append(chunk)
            dic_y[person_id].append(label)

    element_counts = Counter(labels)

    for element, count in element_counts.items():
        print(f"Element {element} occurs {count} times")

    print(len(dic_X[person_id]), len(dic_y[person_id]))

  0%|          | 0/10 [00:00<?, ?it/s]

Id: 038441c925bb


 10%|█         | 1/10 [00:03<00:28,  3.12s/it]

Element 0 occurs 3134 times
Element 2 occurs 12 times
Element 1 occurs 1968 times
Element 3 occurs 15 times
Element 4 occurs 1369 times
5129 5129
Id: 03d92c9f6f8a


 20%|██        | 2/10 [00:09<00:38,  4.81s/it]

...

Element 0 occurs 5241 times
Element 1 occurs 2065 times
Element 2 occurs 13 times
Element 3 occurs 19 times
7338 7338
Id: 0a96f4993bd7


 90%|█████████ | 9/10 [00:34<00:03,  3.45s/it]

Element 0 occurs 2623 times
Element 1 occurs 1639 times
Element 3 occurs 11 times
Element 2 occurs 8 times
4281 4281
Id: 0cd1e3d0ed95


100%|██████████| 10/10 [00:37<00:00,  3.76s/it]

Element 0 occurs 3383 times
Element 2 occurs 2 times
Element 1 occurs 586 times
Element 3 occurs 5 times
Element 4 occurs 2195 times
3976 3976


In [6]:
for key, value in dic_y.items():
    y = value
    X = dic_X[key]

    X_awake = []
    y_awake = []
    X_sleep = []
    y_sleep = []
    X_onset = []
    y_onset = []
    X_wakeup = []
    y_wakeup = []

    for idx in range(len(y)):
        if y[idx] == 0:
            X_awake.append(X[idx])
            y_awake.append(y[idx])
        elif y[idx] == 1:
            X_sleep.append(X[idx])
            y_sleep.append(y[idx])
        elif y[idx] == 2:
            X_onset.append(X[idx])
            y_onset.append(y[idx])
        elif y[idx] == 3:
            X_wakeup.append(X[idx])
            y_wakeup.append(y[idx])

    num_sample = max(len(X_onset), len(X_wakeup))

    indices = random.sample(range(len(X_awake)), num_sample)
    selected_X_awake = [X_awake[i] for i in indices]
    selected_y_awake = [y_awake[i] for i in indices]
    X_awake = selected_X_awake
    y_awake = selected_y_awake

    indices = random.sample(range(len(X_sleep)), num_sample)
    selected_X_sleep = [X_sleep[i] for i in indices]
    selected_y_sleep = [y_sleep[i] for i in indices]
    X_sleep = selected_X_sleep
    y_sleep = selected_y_sleep

    print(len(X_awake), len(X_sleep), len(X_onset), len(X_wakeup))

    X_total = []
    y_total = []
    X_total = X_awake + X_sleep + X_onset + X_wakeup
    y_total = y_awake + y_sleep + y_onset + y_wakeup

    dic_y[key] = y_total
    dic_X[key] = X_total

    print(len(dic_X[key]), len(dic_y[key]))

15 15 12 15
57 57
5 5 5 4
19 19
13 13 10 13
49 49
13 13 13 12
51 51
6 6 4 6
22 22
2 2 2 1
7 7
11 11 11 7
40 40
19 19 13 19
70 70
11 11 8 11
41 41
5 5 2 5
17 17


In [7]:
class MyDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        inp = self.inputs[index]
        output = self.outputs[index]

        input_tensor = torch.tensor(inp, dtype=torch.float32)
        output_tensor = torch.tensor(output, dtype=torch.long)

        return input_tensor, output_tensor

In [8]:
class TransformerModel(nn.Module):
    def __init__(
        self,
        input_dim,
        embed_dim,
        num_classes=4,
        num_layers=8,
        nhead=4,
        dim_feedforward=64,
        dropout=0.2
    ):

        super(TransformerModel, self).__init__()

        self.conv1d = nn.Conv1d(
            in_channels=input_dim, 
            out_channels=embed_dim, 
            kernel_size=3, 
            padding=1
        )
        self.embed_layer = nn.Linear(embed_dim, embed_dim)
        self.layernorm = nn.LayerNorm(embed_dim)

        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=embed_dim,
                nhead=nhead,
                dim_feedforward=dim_feedforward,
                dropout=dropout
            ),
            num_layers=num_layers
        )

        self.classification = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1d(x)
        x = x.permute(0, 2, 1)

        x = self.embed_layer(x)
        x = self.layernorm(x)
        x = self.encoder(x.permute(1, 0, 2))
        x = x[-1]
        x = self.classification(x)
        return x

# 示例
model = TransformerModel(input_dim=2, embed_dim=32)
input_data = torch.rand(3, 360, 2)
output = model(input_data)
print(model)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")
print(output.shape)

TransformerModel(
  (conv1d): Conv1d(2, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (embed_layer): Linear(in_features=32, out_features=32, bias=True)
  (layernorm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_features=64, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=64, out_features=32, bias=True)
        (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (classification): Linear(in_features=32, out_features=4, bias=True)
)
Total 

In [9]:
model = TransformerModel(
    input_dim=CFG.input_dim,
    embed_dim=CFG.embed_dim,
    num_classes=CFG.num_classes,
    num_layers=CFG.num_layers,
    nhead=CFG.nhead,
    dim_feedforward=CFG.dim_feedforward,
    dropout=CFG.dropout
)

criterion = nn.CrossEntropyLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"

total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

Total number of parameters: 35652


In [10]:
# train_series = pd.read_parquet("./train_series.parquet")

# def inference(person_id, model):
#     df = train_series[train_series["series_id"] == person_id]

#     X = []
#     y = []
#     for j in range(0, len(df), CFG.window_overlap):
#         start = j
#         if j + CFG.window_size >= len(df):
#             start = len(df) - CFG.window_size
#             end = len(df)
#             break
#         else:
#             end = j + CFG.window_size
            
#         chunk = df.iloc[start:end][["anglez", "enmo"]]
#         chunk = chunk.to_numpy()
#         X.append(chunk)
#         y.append(-1)

#     dataset = MyDataset(X, y)
#     dataloader = DataLoader(dataset, batch_size=CFG.batch_size, shuffle=False)
    
#     for batch_idx, (data, target) in tqdm(enumerate(dataloader), total=len(dataloader)):

#             data = data.to(device)
#             target = target.to(device)
#             output = model(data)
#             _, predicted = torch.max(output, 1)
#             print(predicted)


In [11]:
train_losses = []
train_step_losses = []
test_losses = []
learning_rates = []

indices = list(range(CFG.num_person))
kf = KFold(n_splits=CFG.fold, shuffle=True, random_state=2023)

for fold, (train_index, val_index) in enumerate(kf.split(indices)):
    # print(f"Fold {fold + 1}")
    print("Train indices:", train_index)
    print("Test indices:", val_index)
    X_train = []
    y_train = []
    for idx in train_index:
        person_id = index_to_ids[idx]
        X_train.extend(dic_X[person_id])
        y_train.extend(dic_y[person_id])

    X_val = []
    y_val = []
    for idx in val_index:
        person_id = index_to_ids[idx]
        X_val.extend(dic_X[person_id])
        y_val.extend(dic_y[person_id])
    
    print(len(X_train), len(X_val))

    train_dataset = MyDataset(X_train, y_train)
    val_dataset = MyDataset(X_val, y_val)
    train_dataloader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)

    train_model = model.to(device=device)
    train_model.train()

    optimizer = torch.optim.Adam(train_model.parameters(), lr=CFG.learning_rate)
    scheduler = StepLR(optimizer, step_size=CFG.scheduler_step_size, gamma=CFG.scheduler_gamma)

    min_loss = np.inf

    for epoch in range(1, CFG.epochs + 1):
        train_loss = 0.0
        test_loss = 0.0
        train_step_loss = 0.0

        learning_rate = optimizer.param_groups[0]["lr"]
        learning_rates.append(learning_rate)

        for batch_idx, (data, target) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
            optimizer.zero_grad()

            data = data.to(device)
            target = target.to(device)
            output = train_model(data)

            loss = criterion(output, target)
            train_loss += loss.item()
            train_step_loss += loss.item()

            loss.backward()
            optimizer.step()

            if (batch_idx + 1) % CFG.train_record_steps == 0:
                train_step_losses.append(train_step_loss / (batch_idx + 1))
                print(f"Fold: {fold + 1}, Epoch: {epoch}, Step: {(batch_idx + 1)}, Train Loss Now: {train_step_loss / (batch_idx + 1)}")

        scheduler.step()

        with torch.no_grad():
            for batch_idx, (data, target) in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
                data = data.to(device)
                target = target.to(device)
                output = train_model(data)

                loss = criterion(output, target)
                test_loss += loss.item()

        train_loss /= len(train_dataloader)
        train_losses.append(train_loss)

        test_loss /= len(val_dataloader)
        test_losses.append(test_loss)

        print(f"Fold: {fold + 1}, Epoch: {epoch}, Train Loss: {train_loss}, Test Loss: {test_loss}")

        if test_loss < min_loss:
            min_loss = test_loss
            torch.save(train_model.state_dict(), f"{CFG.model_dir}/model_epoch_{fold + 1}.pth")

Fold 1
Train indices: [0 1 3 4 5 6 7 9]
Test indices: [2 8]
283 90


 22%|██▏       | 2/9 [00:00<00:00, 19.81it/s]

Step: 1, Train Loss Now: 1.7223035097122192
Step: 2, Train Loss Now: 1.6250168681144714
Step: 3, Train Loss Now: 1.5402263800303142
Step: 4, Train Loss Now: 1.5234940946102142
Step: 5, Train Loss Now: 1.4902157068252564


 67%|██████▋   | 6/9 [00:00<00:00, 27.92it/s]

Step: 6, Train Loss Now: 1.486708442370097
Step: 7, Train Loss Now: 1.4706545897892542
Step: 8, Train Loss Now: 1.4560060054063797


...

100%|██████████| 10/10 [00:00<00:00, 23.61it/s]


Step: 8, Train Loss Now: 0.4201382100582123
Step: 9, Train Loss Now: 0.4164790478017595
Step: 10, Train Loss Now: 0.40866848826408386


 10%|█         | 1/10 [00:00<00:00, 11.37it/s]

Step: 1, Train Loss Now: 0.3065469264984131


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label="Train Loss")
plt.plot(test_losses, label="Test Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train & Test Loss")
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(learning_rates)
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")

plt.savefig("plot.png")

plt.tight_layout()
plt.show()